**Import all the required libraries**

In [55]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import statistics

**The data consists of certain data points such as GrossMargin,WorkingCapitalRatio,EarningPerShare for all the companies. Using these data points, we calculate a variety ratios using mathematical formulae based on the data points. Finally, we have labels (with value 0 or 1) corresponding to these ratios depending on their actual value in comparison to an ideal value it should have.**

**Import the data, and create a column 'aggregate' which stores the mean of all the label columns**

In [56]:
data = pd.read_csv("labels2.csv")
data = data.set_index('cik_date')
data[['wce_label','eps_label','de_label','pe_label','roe_label','growth_rate_label','profitm_label','grossm_label','ro40_label','churnrate_label','EVbyEbidta_label','marketCap_label','magicNum_label']] = data[['wce_label','eps_label','de_label','pe_label','roe_label','growth_rate_label','profitm_label','grossm_label','ro40_label','churnrate_label','EVbyEbidta_label','marketCap_label','magicNum_label']].fillna(0)
data['aggregate'] = data[['wce_label','eps_label','de_label','pe_label','roe_label','growth_rate_label','profitm_label','grossm_label','ro40_label','churnrate_label','EVbyEbidta_label','marketCap_label','magicNum_label']].mean(axis = 1, skipna = True)

**We assign average value of the column to all the missing data points**

In [57]:
avg_values = data.mean(axis = 0, skipna = True).fillna(0).to_dict()
for key in data.keys():
  data[key] = data[key].fillna(avg_values[key])

***For training our model, the input features are the data points and calculated ratios, while the output feature is the 'aggregate' column which is the sum of all the label columns***

In [58]:
X = data[['GrossProfit','GrossMargin','WorkingCapitalRatio','EarningPerShare','DebtToEquityRatio','PEratio','ReturnOfEquity','EBIDTAratio','EvRatio','EVbyEbidta','ChurnRate','GrowthRate','ProfitMargin','RuleOf40','MarketCap','MagicNumber']]
y = data['aggregate']

In [59]:
print(statistics.mean(list(y)))

0.3060320973990039


***Split the dataset into train and test***

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [61]:
X_train.shape , X_test.shape , y_train.shape , y_test.shape

((97, 16), (42, 16), (97,), (42,))

**We perform regression on the 'aggregate' value**

In [62]:
from xgboost import XGBRegressor

xgb_regressor = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

xgb_regressor.fit(X_train,y_train)
xgb_test_preds = xgb_regressor.predict(X_test)

[07:02:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [75]:
from sklearn.metrics import mean_squared_error
error = mean_squared_error(list(y_test), xgb_test_preds)
print(error)

0.003685199672335579


In [74]:
final_preds = xgb_regressor.predict(X)
data['predicted_aggregate'] = final_preds
data.to_csv('predicted_company_scores.csv')

In [76]:
import pickle
with open('xgbmodel_regression.pkl', 'wb') as fid:
    pickle.dump(xgb_regressor,fid,2) 

**Based on the output of the regression model, we generate labels 0,1,2 using some threshold values**

In [64]:
def findLabel(x):
  if x>=0 and x<=0.2:
    return 0
  elif x>0.2 and x<=0.4:
    return 1
  else:
    return 2

In [65]:
test_labels = list()
pred_labels = list()

In [66]:
count = 0
for ind in range(len(list(y_test.to_dict().values()))):
  test_score = y_test[ind]
  pred_score = xgb_test_preds[ind]
  test_labels.append(findLabel(test_score))
  pred_labels.append(findLabel(pred_score))
  count+=1
print(count)

42


In [67]:
print(test_labels)

[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 2, 2, 0, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1]


In [68]:
print(pred_labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 0, 1, 1, 2, 1, 1, 1, 0, 2, 0, 2, 1, 0, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1]


In [69]:
print(len(test_labels),len(pred_labels))

42 42


**We calculate the final labels using the regression outputs and the threshold value**

In [70]:
from sklearn.metrics import accuracy_score , precision_score, recall_score, f1_score 

accuracy = accuracy_score(test_labels,pred_labels)
precision = precision_score(test_labels,pred_labels,average=None)
recall = recall_score(test_labels,pred_labels,average=None)
f1 = f1_score(test_labels,pred_labels,average=None)


print(f"accuracy: {accuracy}")
print(f"precision_score: {precision}")
print(f"recall_score: {recall}")
print(f"f1_score: {f1}")

accuracy: 0.7619047619047619
precision_score: [0.83333333 0.81481481 0.55555556]
recall_score: [0.625      0.81481481 0.71428571]
f1_score: [0.71428571 0.81481481 0.625     ]
